<a href="https://colab.research.google.com/github/Ape108/Quantum_Portfolio_Optimization/blob/main/MVP_Completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Optimization using QAOA

This notebook demonstrates how to solve a cardinality-constrained portfolio optimization problem using the Quantum Approximate Optimization Algorithm (QAOA).

The process involves these key steps:
1.  **Data Acquisition**: Fetching the top holdings of a target ETF (XLK) and their historical price data.
2.  **Financial Modeling**: Calculating the expected annual returns (`mu`) and the covariance matrix (`Sigma`) for the assets.
3.  **QUBO Formulation**: Translating the financial problem (maximize return, minimize risk) into a Quadratic Unconstrained Binary Optimization (QUBO) matrix `Q`. This includes a penalty term to enforce the constraint on the number of assets to select.
4.  **Quantum Solution**: Using Qiskit's QAOA implementation to find the bitstring that minimizes the QUBO, which corresponds to the optimal portfolio selection.
5.  **Analysis**: Running the QAOA with an increasing number of repetitions (`reps`) to analyze how the solution quality improves.

## 1. Setup and Environment

This section handles the initial setup, including installing necessary libraries and importing the required modules from standard Python, third-party data science libraries, and Qiskit.

In [ ]:
!pip install qiskit

In [ ]:
# ---- Standard library ----
import os
import re
import io
import time
import subprocess
import shlex

# ---- Third-party libraries ----
import numpy as np
import pandas as pd
import requests
import yfinance as yf
from collections import defaultdict

# ---- Quantum Circuit ----
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit.library import QAOAAnsatz
from qiskit.visualization import plot_histogram

# ---- Colab/Google ----
from google.colab import drive

# ---- Warnings ----
import warnings
from scipy.sparse import SparseEfficiencyWarning

# This will ignore all SparseEfficiencyWarning messages
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)


## 2. Data Acquisition and Financial Modeling

Here, we define our target ETF and fetch the necessary financial data.

- **Fetch Holdings**: We start by scraping the top 5 holdings for the Technology Select Sector SPDR Fund (XLK) from the SSGA website.
- **Download Price History**: Using `yfinance`, we download 1-minute interval price data for the last 7 days for these tickers.
- **Calculate Returns and Covariance**: From the price data, we compute the percentage change to get minute-by-minute returns. These are then annualized to produce our key financial inputs:
    - `mu`: The vector of expected annual returns for each asset.
    - `Sigma`: The annualized covariance matrix, which represents the risk and correlation between the assets.

In [ ]:
def fetch_holdings_from_ssga(symbol: str) -> pd.DataFrame:
    """
    Return a clean DataFrame of holdings for an SSGA ETF.
    This is a simplified version that assumes a predictable Excel format.
    """
    # 1. Download the Excel file into an in-memory buffer
    url = f"https://www.ssga.com/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{symbol.lower()}.xlsx"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    excel_file = io.BytesIO(response.content)

    # 2. Find the header row by searching for 'Ticker' or 'Symbol'
    # We read the first 40 rows just to inspect them
    temp_df = pd.read_excel(excel_file, header=None, dtype=str, nrows=40)
    header_row_index = None
    for i, row in temp_df.iterrows():
        # Check if any cell in the row contains 'Ticker' or 'Symbol' (case-insensitive)
        if row.str.contains(r'^(ticker|symbol)$', case=False, na=False).any():
            header_row_index = i
            break

    if header_row_index is None:
        raise RuntimeError("Could not find the header row in the Excel file.")

    # 3. Read the Excel file again, this time with the correct header
    # The buffer automatically resets, so we can read from it again
    df = pd.read_excel(excel_file, header=header_row_index)

    # 4. Find the correct column for the tickers
    ticker_col_candidates = ['ticker', 'symbol', 'ticker symbol']
    ticker_col = next((col for col in df.columns if str(col).strip().lower() in ticker_col_candidates), None)

    if ticker_col is None:
        raise RuntimeError(f"Could not find a ticker column. Found columns: {list(df.columns)}")

    # 5. Extract, clean, and format the ticker data into a final DataFrame
    tickers = df[ticker_col].dropna().astype(str)

    # Filter for strings that look like valid tickers
    valid_ticker_mask = tickers.str.match(r"^[A-Z0-9]{1,6}([.-][A-Z0-9]{1,3})?$", case=False)

    clean_tickers = (
        tickers[valid_ticker_mask]
        .str.upper()
        .str.replace("-", ".", regex=False) # Standardize tickers like 'BRK-B' to 'BRK.B'
        .drop_duplicates()
    )

    return pd.DataFrame({"Ticker": clean_tickers}).reset_index(drop=True)




In [ ]:
PRICE_COLS = ["Open","High","Low","Close","Adj Close","Volume"]
ETF_SYMBOL = "XLK"
tickers = fetch_holdings_from_ssga(ETF_SYMBOL).head(5) # Grab 5 tickers
tickers.head(5)

/tmp/ipython-input-961546454.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if row.str.contains(r'^(ticker|symbol)$', case=False, na=False).any():


,Ticker
0,NVDA
1,MSFT
2,AAPL
3,AVGO
4,PLTR


In [ ]:
# --- Make sure tickers is a list of strings ---
# If you started with something like weights_df["Ticker"] or another DataFrame/Series:
if isinstance(tickers, pd.DataFrame):
    tickers = tickers.squeeze()
if isinstance(tickers, pd.Series):
    tickers = tickers.dropna().astype(str).unique().tolist()
elif isinstance(tickers, str):
    tickers = [tickers]  # allow single string
else:
    tickers = list(map(str, tickers))  # e.g., set/tuple -> list[str]

# --- Download 1-minute data for the last 7 days ---
data = yf.download(
    tickers,
    period="7d",
    interval="1m",
    auto_adjust=True,
    group_by="ticker",   # for multiple tickers, returns a dict-like column structure
    threads=True
)

# --- Build a Close-price DataFrame (handles 1 or many tickers) ---
if len(tickers) == 1:
    # Single ticker: columns like ['Open','High','Low','Close',...]
    close_df = data[['Close']].rename(columns={'Close': tickers[0]})
else:
    # Multiple tickers: columns like data['AAPL']['Close'], etc.
    # Some tickers might be missing (if yfinance couldn't fetch); guard for that.
    pieces = {}
    for t in tickers:
        if (t in data) and ('Close' in data[t].columns):
            pieces[t] = data[t]['Close']
    if not pieces:
        raise RuntimeError("No Close data retrieved for any tickers.")
    close_df = pd.concat(pieces, axis=1)

# --- 1-minute simple returns ---
minute_returns = close_df.pct_change().dropna(how="all")

print("\n--- Sample of 1-minute returns ---")
print(minute_returns.head())

# --- Annualization (per-minute to per-year) ---
MIN_PER_DAY = 390        # US regular session minutes
TRADING_DAYS = 252
MIN_PER_YEAR = MIN_PER_DAY * TRADING_DAYS

# Expected returns vector (mu) and covariance matrix (Sigma)
mu = minute_returns.mean() * MIN_PER_YEAR
Sigma = minute_returns.cov() * MIN_PER_YEAR

print("\n--- mu (annualized) ---")
print(mu.head() if hasattr(mu, "head") else mu)
print("\n--- Sigma (annualized, top-left 5x5) ---")
print(Sigma.iloc[:5, :5])


[*********************100%***********************]  5 of 5 completed


--- Sample of 1-minute returns ---
                               NVDA      MSFT      AAPL      AVGO      PLTR
Datetime                                                                   
2025-09-24 13:31:00+00:00 -0.001143  0.001291 -0.001311 -0.001703 -0.001030
2025-09-24 13:32:00+00:00 -0.001284 -0.001523 -0.001943 -0.006600  0.002128
2025-09-24 13:33:00+00:00 -0.002880 -0.002934  0.000295 -0.001165 -0.001688
2025-09-24 13:34:00+00:00  0.002104 -0.000942 -0.003012  0.003946 -0.000818
2025-09-24 13:35:00+00:00  0.001846 -0.000491  0.001798  0.001087  0.000873

--- mu (annualized) ---
NVDA    1.920310
MSFT    0.310547
AAPL    0.306469
AVGO    0.119567
PLTR    0.765148
dtype: float64

--- Sigma (annualized, top-left 5x5) ---
          NVDA      MSFT      AAPL      AVGO      PLTR
NVDA  0.094550  0.009024  0.003873  0.063362  0.074287
MSFT  0.009024  0.027760  0.003797  0.011723  0.007846
AAPL  0.003873  0.003797  0.032029  0.006864  0.004894
AVGO  0.063362  0.011723  0.006864  0.135070 

## 3. QUBO Formulation

To solve this problem on a quantum computer, we must formulate it as a QUBO. The goal is to find a binary vector `x` where `x_i = 1` if we select the asset and `0` otherwise.

The objective function to be minimized is:

$ \min_{x \in \{0,1\}^n} \left( x^T \Sigma x - q \mu^T x \right) + P \left( \sum_{i=1}^n x_i - K \right)^2 $

This function consists of three parts:
1.  **Risk Term ($x^T \Sigma x$)**: The portfolio variance, which we want to minimize.
2.  **Return Term ($-q \mu^T x$)**: The negated expected return. The parameter `q` balances the trade-off between maximizing return and minimizing risk.
3.  **Constraint Penalty ($P (\sum x_i - K)^2$)**: A penalty term that ensures the total number of selected assets is exactly equal to our target `K`. The penalty coefficient `P` must be large enough to make violating the constraint undesirable for the optimizer.

The functions below build the corresponding QUBO matrix `Q` from `mu`, `Sigma`, `K`, `q`, and `P`.

In [ ]:

def build_qubo_Q(tickers, Sigma: pd.DataFrame, mu: pd.Series, K: int = 3, q: float = 0.5, P: float = 10.0):
    """
    Build a QUBO Q matrix for cardinality-constrained portfolio selection.

    Args:
        tickers: list-like of tickers, defines variable order (x_i for tickers[i])
        Sigma: covariance matrix (pd.DataFrame) indexed/columned by tickers
        mu: expected returns (pd.Series) indexed by tickers
        K: number of assets to select
        q: risk/return tradeoff (higher puts more weight on returns term)
        P: penalty weight for enforcing sum(x_i)=K

    Returns:
        Q: defaultdict(float) with keys (i,j) in 0..n-1
        index_of: dict mapping ticker -> index
    """
    tickers = list(tickers)
    num_assets = len(tickers)
    index_of = {t: i for i, t in enumerate(tickers)}

    # align Sigma and mu to the given tickers order
    Sigma_aligned = Sigma.loc[tickers, tickers]
    mu_aligned = mu.loc[tickers]

    Q = defaultdict(float)

    # 1) Risk term (covariance)
    for i in range(num_assets):
        for j in range(num_assets):
            Q[(i, j)] += float(Sigma_aligned.iloc[i, j])

    # 2) Return term (negated for minimization)
    for i in range(num_assets):
        Q[(i, i)] -= float(q) * float(mu_aligned.iloc[i])

    # 3) Constraint penalty: (sum x_i - K)^2 = (1 - 2K)*sum x_i + 2*sum_{i<j} x_i x_j + const
    for i in range(num_assets):
        Q[(i, i)] += float(P) * (1 - 2 * K)
        for j in range(i + 1, num_assets):
            Q[(i, j)] += float(P) * 2.0

    return Q, index_of

# ---------- Example usage (mirrors your snippet) ----------
# # user parameters
# K = 3
# q = 0.5
# P = 1.0
#
# tickers = xlk_top_ten  # your list
# Q, index_of = build_qubo_Q(tickers, Sigma=Sigma, mu=mu, K=K, q=q, P=P)

In [ ]:
def qubo_to_matrix_df(Q_dict, tickers):
    n = len(tickers)
    M = np.zeros((n, n), dtype=float)
    for (i, j), v in Q_dict.items():
        M[i, j] += float(v)
    # symmetrize (xᵀQx expects a symmetric Q)
    M = (M + M.T) / 2.0
    return pd.DataFrame(M, index=tickers, columns=tickers)

def normalize_qubo_df(Q_df, limit=1.0):
    """Scale Q so that max |entry| == limit. Returns (Q_scaled, scale)."""
    m = Q_df.abs().to_numpy().max()
    if m == 0:
        return Q_df.copy(), 1.0
    s = limit / m
    return Q_df * s, s


In [ ]:
K = 2
q = 0.5
P = 10.0

Q_dict, index_of = build_qubo_Q(tickers, Sigma, mu, K=K, q=q, P=P)
Q_df = qubo_to_matrix_df(Q_dict, tickers)

Q_norm, _ = normalize_qubo_df(Q_df)

Q_norm  # <- this is your matrix version of Q


,NVDA,MSFT,AAPL,AVGO,PLTR
NVDA,-1.000000,0.324278,0.324111,0.326038,0.326392
MSFT,0.324278,-0.976087,0.324108,0.324365,0.324239
AAPL,0.324111,0.324108,-0.975883,0.324208,0.324144
AVGO,0.326038,0.324365,0.324208,-0.969516,0.326122
PLTR,0.326392,0.324239,0.324144,0.326122,-0.979499


## 4. Solving with QAOA

The QUBO problem is solved using QAOA. This involves a few helper functions to manage the conversion and execution.

- **QUBO to Ising Hamiltonian**: The `qubo_to_ising_sparsepauli` function converts the QUBO matrix `Q` into an equivalent Ising Hamiltonian `H` and an energy offset. QAOA works by finding the ground state (minimum energy) of this Hamiltonian.
- **QAOA Driver**: The `run_qaoa_qubo` function orchestrates the entire process. It takes the QUBO matrix, creates a `QAOAAnsatz` circuit, and uses a classical optimizer (`L-BFGS-B`) to find the optimal parameters (`gamma` and `beta`) that prepare the ground state.
- **Statevector Simulation**: For stability and accuracy, the objective function is evaluated using a noiseless statevector simulator. The final result is determined by finding the most probable bitstring in the final optimized statevector.

In [ ]:

def ensure_Q_df(Q_df):
    # symmetrize just in case of tiny asymmetries
    Q_df = (Q_df + Q_df.T) / 2.0
    return Q_df

def qubo_to_ising_sparsepauli(Q_df):
    """
    Map QUBO min x^T Q x (x in {0,1}^n) to Ising:
      x_i = (1 - z_i)/2  with z_i in {-1, +1}
    Returns SparsePauliOp H and an energy offset (float).
    """
    Q = Q_df.to_numpy(copy=True)
    n = Q.shape[0]

    # build coefficients
    J_upper = {}  # (i,j)->coeff for i<j
    h = np.zeros(n)
    offset = 0.0

    # off-diagonal: J_ij = 1/4 * Q_ij, and offset += 1/4 Q_ij
    for i in range(n):
        for j in range(i+1, n):
            Jij = 0.25 * Q[i, j]
            if Jij != 0.0:
                J_upper[(i, j)] = Jij
            offset += 0.25 * Q[i, j]

    # diagonal: h_i = -1/2 Q_ii - 1/4 * sum_{j!=i} Q_ij, and offset += 1/2 Q_ii
    row_sums = Q.sum(axis=1)
    for i in range(n):
        h[i] = -0.5 * Q[i, i] - 0.25 * (row_sums[i] - Q[i, i])
        offset += 0.5 * Q[i, i]

    # build SparsePauliOp H = sum_i h_i Z_i + sum_{i<j} J_ij Z_i Z_j
    # note: Qiskit qubit 0 is rightmost in the label string; we’ll place Z in the right spots.
    labels = []
    coeffs = []

    # single-Z terms
    for i in range(n):
        if h[i] != 0.0:
            z = ["I"] * n
            z[n - 1 - i] = "Z"  # reverse index for label
            labels.append("".join(z))
            coeffs.append(float(h[i]))

    # ZZ terms
    for (i, j), Jij in J_upper.items():
        if Jij != 0.0:
            z = ["I"] * n
            z[n - 1 - i] = "Z"
            z[n - 1 - j] = "Z"
            labels.append("".join(z))
            coeffs.append(float(Jij))

    H = SparsePauliOp.from_list(list(zip(labels, coeffs))) if labels else SparsePauliOp.from_list([("I"*n, 0.0)])
    return H, float(offset)

def bitstring_from_statevector_probs(probs_dict):
    """Return the most likely bitstring and its prob from a probs dict."""
    if not probs_dict:
        return None, 0.0
    items = sorted(probs_dict.items(), key=lambda kv: kv[1], reverse=True)
    return items[0][0], items[0][1]


In [ ]:

def run_qaoa_qubo(Q_df, reps=2, shots=4096, seed=42):
    """
    QAOA on a QUBO given as symmetric DataFrame Q_df.
    Uses Statevector for objective evaluation (stable across environments).
    Returns (best_bits, best_val, result_dict)
    """
    rng = np.random.default_rng(seed)
    Q_df = ensure_Q_df(Q_df)
    tickers = list(Q_df.index)
    n = len(tickers)

    # QUBO -> Ising (SparsePauliOp) + offset (we'll track it for reporting)
    H, offset = qubo_to_ising_sparsepauli(Q_df)

    # QAOA ansatz
    ansatz = QAOAAnsatz(cost_operator=H, reps=reps)  # mixer is default X-mixer
    print(ansatz)

    # Decompose the Circuit
    decomposed_ansatz = ansatz.decompose()
    print("--- Decomposed QAOA Circuit ---")
    print(decomposed_ansatz)


    # parameter vector: order in QAOAAnsatz is [β_0..β_{p-1}, γ_0..γ_{p-1}]
    param_vec = ansatz.parameters
    d = len(param_vec)
    x0 = rng.uniform(-np.pi, np.pi, size=d)

    # objective: <H> + offset
    def objective(theta):
        bound = ansatz.assign_parameters(dict(zip(param_vec, theta)), inplace=False)
        psi = Statevector.from_instruction(bound)
        val = psi.expectation_value(H).real + offset
        return float(val)

    # local optimize
    bounds = [(-np.pi, np.pi)] * d
    res = minimize(objective, x0, method="L-BFGS-B", bounds=bounds, options={"maxiter": 300})

    # best expectation (energy)
    best_val = float(res.fun)

    # build final state and measurement distribution (via Statevector)
    final_circ = ansatz.assign_parameters(dict(zip(param_vec, res.x)), inplace=False)
    sv = Statevector.from_instruction(final_circ)
    probs = sv.probabilities_dict()  # bitstring -> prob (little-endian)

    # choose best bitstring by probability (common practice for QAOA readout)
    best_bits, best_prob = bitstring_from_statevector_probs(probs)

    # pretty plot
    fig = plot_histogram(probs, title="QAOA Bitstring Distribution (statevector)")
    plt.show()

    return {
        "energy": best_val,
        "bits": best_bits,
        "prob": best_prob,
        "probs": probs,
        "params": res.x,
        "tickers": tickers,
    }

## 5. Experiment and Analysis

Finally, we run the experiment.

We print:
- The approximate ground energy found by the algorithm.
- The most probable bitstring solution (`1` indicates a selected asset).
- The list of ticker symbols corresponding to the optimal portfolio.

In [ ]:
num_reps = 1

result = run_qaoa_qubo(Q_df, reps=num_reps, shots=4096, seed=42)
print("Approx. ground energy (incl. offset):", result["energy"])
print("Most probable bitstring:", result["bits"], f"(p={result['prob']:.3f})")
print("Selected tickers:", [t for t, b in zip(result["tickers"], result["bits"][::-1]) if b == "1"])



     ┌──────────────────┐
q_0: ┤0                 ├
     │                  │
q_1: ┤1                 ├
     │                  │
q_2: ┤2 QAOA(γ[0],β[0]) ├
     │                  │
q_3: ┤3                 ├
     │                  │
q_4: ┤4                 ├
     └──────────────────┘
--- Decomposed QAOA Circuit ---
     ┌───┐»
q_0: ┤ H ├»
     ├───┤»
q_1: ┤ H ├»
     ├───┤»
q_2: ┤ H ├»
     ├───┤»
q_3: ┤ H ├»
     ├───┤»
q_4: ┤ H ├»
     └───┘»
«     ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                        ├»
«     │                                                                                                                                         │»
«q_1: ┤1                                                                                                     